In [1]:
#!pip install rank-bm25
#from rank_bm25 import BM25Okapi
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from math import floor, ceil

import json
import gzip

from os import walk
from scipy.spatial import KDTree

!pip install geopy
!pip install phonenumbers
!pip install pycountry

import geopy.distance
import phonenumbers
import pycountry



In [2]:
#path = r"../src/data/"
#file_path = path + r"/LB_cleaned"

path = r"C:\Users\marvi\Desktop\SchemaOrgData"
file_path = path + r"\geo_preprocessed"
files = os.listdir(file_path)


In [3]:
files

['LocalBusiness_101attorney.com_September2020.json.gz',
 'LocalBusiness_101dentist.com_September2020.json.gz',
 'LocalBusiness_11880.com_September2020.json.gz',
 'LocalBusiness_2findlocal.com_September2020.json.gz',
 'LocalBusiness_99insurance.com_September2020.json.gz',
 'LocalBusiness_alltrails.com_September2020.json.gz',
 'LocalBusiness_attorneyhelp.org_September2020.json.gz',
 'LocalBusiness_aussieweb.com.au_September2020.json.gz',
 'LocalBusiness_bankbazaar.com_September2020.json.gz',
 'LocalBusiness_birdeye.com_September2020.json.gz',
 'LocalBusiness_chambermaster.com_September2020.json.gz',
 'LocalBusiness_churchangel.com_September2020.json.gz',
 'LocalBusiness_civico.com_September2020.json.gz',
 'LocalBusiness_countyoffice.org_September2020.json.gz',
 'LocalBusiness_dandb.com_September2020.json.gz',
 'LocalBusiness_divorcemag.com_September2020.json.gz',
 'LocalBusiness_enyakinnerde.com_September2020.json.gz',
 'LocalBusiness_familiesonline.co.uk_September2020.json.gz',
 'LocalB

In [4]:
df_all = pd.DataFrame()
for file in files:
    df = pd.read_json(os.path.join(file_path, file), compression='gzip', lines=True)
    df['table id'] = file.strip('.json.gz')
    df_all = pd.concat([df_all, df], ignore_index=True)

In [5]:
len(df_all)

574925

## Only get entries where dataframe has non-zero telephone numbers AND non-zero country codes

In [6]:
df_clean = df_all[~df_all["addresscountry"].isna()]
df_clean = df_clean[~df_clean["telephone"].isna()]
len(df_clean)

164220

In [7]:
lat = 'latitude'
lon = 'longitude'

df_clean = df_clean[df_clean[lon].notna()]
df_clean = df_clean[df_clean[lat].notna()]
len(df_clean)

96071

### Format longitude and latitude

In [8]:
longArray = df_clean[lon].to_numpy().astype(str)
longArray = np.char.replace(longArray, ',', '.')
df_clean[lon] = longArray
df_clean[lon] = df_clean[lon].astype(float)

latArray = df_clean[lat].to_numpy().astype(str)
latArray = np.char.replace(latArray, ',', '.')
df_clean[lat] = latArray
df_clean[lat] = df_clean[lat].astype(float)

In [9]:
pd.options.display.max_columns = 100
df_clean[:3]

,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname
21375,7,Fuchsberger Immobilien,"{'addresscountry': 'DE', 'streetaddress': 'Obe...",(06104)&nbsp;43864,https://www.11880.com/branchenbuch/obertshause...,Oberweiser Str. 7,Hessen,63179,Obertshausen,LocalBusiness_11880.com_September2020,None,info@fuchsberger-immobilien.de,"{'latitude': '50,073592', 'longitude': '8,84146'}",http://www.fuchsberger-immobilien.de,[https://cdn.11880.com/fuchsberger-immobilien_...,"[{'closes': '20:00', 'dayofweek': 'Mittwoch', ...",(06104) 499515,None,https://cdn.11880.com/fuchsberger-immobilien_2...,2020-09-18,DE,NaN,50.073592,8.841460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21380,19,Ristorante Da Capo,"{'addresslocality': 'Krefeld', 'addresscountry...",(02151)&nbsp;734710,https://www.11880.com/branchenbuch/krefeld/071...,Hülser Markt 10,Nordrhein-Westfalen,47839,Krefeld,LocalBusiness_11880.com_September2020,None,ristorantedacapo@hotmail.de,"{'latitude': '51,372843', 'longitude': '6,5092...",http://dacaporistorante.com,https://cdn.11880.com/ristorante-da-capo_24583...,"[{'opens': ['18:00', '12:00'], 'closes': ['23:...",None,None,https://cdn.11880.com/ristorante-da-capo_24583...,2020-09-27,DE,NaN,51.372843,6.509248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21381,20,Bäckerei Mareis GmbH,"{'addresscountry': 'DE', 'addressregion': 'Bay...",(0871)&nbsp;95389680,https://www.11880.com/branchenbuch/landshut-is...,Ritter-von-Schoch-Str. 20,Bayern,84036,Landshut,LocalBusiness_11880.com_September2020,"{'ratingvalue': '4.00', 'bestrating': '5', 'wo...",info@mareis.com,"{'longitude': '12,179405', 'latitude': '48,546...",https://www.mareis.com,//static.11880.com/Portal/images/11880/photo-u...,"[{'closes': '20:00', 'opens': '06:00', 'dayofw...",None,"{'datemodified': '2020-06-29', 'description': ...",None,2020-09-30,DE,NaN,48.546776,12.179405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Second preprocessing step

### Remove non-digits from telephone numbers

In [10]:
def remove_non_digits(string):
    cleaned = re.sub('[^0-9]','', string)
    return cleaned

In [11]:
df_clean['telephone_'] = df_clean['telephone'].astype('str').apply(remove_non_digits)

### Extract country codes to ISO-2 format using ``pycountry``

In [12]:
input_countries = df_clean["addresscountry"]

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

countries


{'Aruba': 'AW',
 'Afghanistan': 'AF',
 'Angola': 'AO',
 'Anguilla': 'AI',
 'Åland Islands': 'AX',
 'Albania': 'AL',
 'Andorra': 'AD',
 'United Arab Emirates': 'AE',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'American Samoa': 'AS',
 'Antarctica': 'AQ',
 'French Southern Territories': 'TF',
 'Antigua and Barbuda': 'AG',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Burundi': 'BI',
 'Belgium': 'BE',
 'Benin': 'BJ',
 'Bonaire, Sint Eustatius and Saba': 'BQ',
 'Burkina Faso': 'BF',
 'Bangladesh': 'BD',
 'Bulgaria': 'BG',
 'Bahrain': 'BH',
 'Bahamas': 'BS',
 'Bosnia and Herzegovina': 'BA',
 'Saint Barthélemy': 'BL',
 'Belarus': 'BY',
 'Belize': 'BZ',
 'Bermuda': 'BM',
 'Bolivia, Plurinational State of': 'BO',
 'Brazil': 'BR',
 'Barbados': 'BB',
 'Brunei Darussalam': 'BN',
 'Bhutan': 'BT',
 'Bouvet Island': 'BV',
 'Botswana': 'BW',
 'Central African Republic': 'CF',
 'Canada': 'CA',
 'Cocos (Keeling) Islands': 'CC',
 'Switzerland': 'CH',
 'Chile': 'CL',
 'China': 'CN',
 "Côte d'Iv

In [13]:
for key, value in countries.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

In [14]:
df_clean["addresscountry"].unique()

array(['DE', 'UNITED STATES', 'CANADA', 'UNITED KINGDOM', 'AUSTRALIA',
       'INDIA', 'UNITED ARAB EMIRATES', 'CL', 'CO', 'MX', 'US', 'GB',
       'TR', 'IN', 'Russia', 'South Korea', 'FR', 'ES', 'PT', 'Vietnam',
       'PL', 'BR', 'ZA', 'AR', 'Taiwan', 'TH', 'NL', 'IT', 'PE', 'JP',
       'LV', 'UA', 'BG', 'Venezuela', 'AE', 'ID', 'CH', 'Saint Helena',
       'SA', 'MY', 'PH', 'EG', 'BE', 'AU', 'Bolivia', 'NI', 'BY', 'CA',
       'MN', 'AT', 'LT', 'SV', 'MA', 'KZ', 'EC', 'ZM', 'PR', 'HK SAR CN',
       'CN', 'SG', 'LB', 'IE', 'JO', 'BZ', 'DZ', 'NL Antilles', 'UG',
       'SE', 'AF', 'AZ', 'PK', 'ZW', 'GR',
       'translation missing: us.countries.GB', 'DMn Republic', 'USA',
       'PUR', ', IN', 'india', 'IN,133006', 'de', 'nz', 'us', 'au', 'gb',
       'es', 'at', 'sg', 'ch', 'ca', 'za', 'ee', 'it', 'dk', 'fr', 'no',
       'pl', 'li', 'lu', 'mx', 'nl', 'br', 'fi', 'se', 'be', 'UY'],
      dtype=object)

## Manually normalize countries which do not exist by default

In [15]:
country_dictionary = {"UNITED STATES": "US", "USA":"US", "UNITED KINGDOM": "GB", "UK": "GB", "CANADA": "CA", "AUSTRALIA": "AU", "UNITED ARAB EMIRATES": "AE" , "INDIA" : "IN", "NEW ZEALAND": "NZ"}
for key, value in country_dictionary.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

In [16]:
df_clean["addresscountry"] = df_clean["addresscountry"].str.upper()

In [17]:
df_clean["addresscountry"].unique()

array(['DE', 'US', 'CA', 'GB', 'AU', 'IN', 'AE', 'CL', 'CO', 'MX', 'TR',
       'RUSSIA', 'SOUTH KOREA', 'FR', 'ES', 'PT', 'VIETNAM', 'PL', 'BR',
       'ZA', 'AR', 'TAIWAN', 'TH', 'NL', 'IT', 'PE', 'JP', 'LV', 'UA',
       'BG', 'VENEZUELA', 'ID', 'CH', 'SAINT HELENA', 'SA', 'MY', 'PH',
       'EG', 'BE', 'BOLIVIA', 'NI', 'BY', 'MN', 'AT', 'LT', 'SV', 'MA',
       'KZ', 'EC', 'ZM', 'PR', 'HK SAR CN', 'CN', 'SG', 'LB', 'IE', 'JO',
       'BZ', 'DZ', 'NL ANTILLES', 'UG', 'SE', 'AF', 'AZ', 'PK', 'ZW',
       'GR', 'TRANSLATION MISSING: US.COUNTRIES.GB', 'DMN REPUBLIC',
       'PUR', ', IN', 'INDIA', 'IN,133006', 'NZ', 'EE', 'DK', 'NO', 'LI',
       'LU', 'FI', 'UY'], dtype=object)

## Remove non-covered countries

In [18]:
df_clean.reset_index(inplace=True)

In [19]:
liste = []
for i, row in enumerate(df_clean["addresscountry"]):
    if len(row) > 2:
        liste.append(i)
        
df_clean = df_clean.drop(liste)        

In [20]:
df_clean["addresscountry"].unique()

array(['DE', 'US', 'CA', 'GB', 'AU', 'IN', 'AE', 'CL', 'CO', 'MX', 'TR',
       'FR', 'ES', 'PT', 'PL', 'BR', 'ZA', 'AR', 'TH', 'NL', 'IT', 'PE',
       'JP', 'LV', 'UA', 'BG', 'ID', 'CH', 'SA', 'MY', 'PH', 'EG', 'BE',
       'NI', 'BY', 'MN', 'AT', 'LT', 'SV', 'MA', 'KZ', 'EC', 'ZM', 'PR',
       'CN', 'SG', 'LB', 'IE', 'JO', 'BZ', 'DZ', 'UG', 'SE', 'AF', 'AZ',
       'PK', 'ZW', 'GR', 'NZ', 'EE', 'DK', 'NO', 'LI', 'LU', 'FI', 'UY'],
      dtype=object)

## Drop empty phonenumbers and too lenghty phone numbers

In [21]:
df_clean = df_clean[df_clean["telephone_"] != "" ]

In [22]:
liste = []
df_clean.reset_index(inplace=True)
for row_index in df_clean.index:
    if len(df_clean.iloc[row_index]["telephone_"])>18:
        liste.append(row_index)

In [23]:
df_clean.drop(labels = liste, inplace = True)

In [24]:
df_clean = df_clean.drop(columns = "index")
df_clean.head()

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_
0,0,7,Fuchsberger Immobilien,"{'addresscountry': 'DE', 'streetaddress': 'Obe...",(06104)&nbsp;43864,https://www.11880.com/branchenbuch/obertshause...,Oberweiser Str. 7,Hessen,63179,Obertshausen,LocalBusiness_11880.com_September2020,None,info@fuchsberger-immobilien.de,"{'latitude': '50,073592', 'longitude': '8,84146'}",http://www.fuchsberger-immobilien.de,[https://cdn.11880.com/fuchsberger-immobilien_...,"[{'closes': '20:00', 'dayofweek': 'Mittwoch', ...",(06104) 499515,None,https://cdn.11880.com/fuchsberger-immobilien_2...,2020-09-18,DE,NaN,50.073592,8.841460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0610443864
1,1,19,Ristorante Da Capo,"{'addresslocality': 'Krefeld', 'addresscountry...",(02151)&nbsp;734710,https://www.11880.com/branchenbuch/krefeld/071...,Hülser Markt 10,Nordrhein-Westfalen,47839,Krefeld,LocalBusiness_11880.com_September2020,None,ristorantedacapo@hotmail.de,"{'latitude': '51,372843', 'longitude': '6,5092...",http://dacaporistorante.com,https://cdn.11880.com/ristorante-da-capo_24583...,"[{'opens': ['18:00', '12:00'], 'closes': ['23:...",None,None,https://cdn.11880.com/ristorante-da-capo_24583...,2020-09-27,DE,NaN,51.372843,6.509248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02151734710
2,2,20,Bäckerei Mareis GmbH,"{'addresscountry': 'DE', 'addressregion': 'Bay...",(0871)&nbsp;95389680,https://www.11880.com/branchenbuch/landshut-is...,Ritter-von-Schoch-Str. 20,Bayern,84036,Landshut,LocalBusiness_11880.com_September2020,"{'ratingvalue': '4.00', 'bestrating': '5', 'wo...",info@mareis.com,"{'longitude': '12,179405', 'latitude': '48,546...",https://www.mareis.com,//static.11880.com/Portal/images/11880/photo-u...,"[{'closes': '20:00', 'opens': '06:00', 'dayofw...",None,"{'datemodified': '2020-06-29', 'description': ...",None,2020-09-30,DE,NaN,48.546776,12.179405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,087195389680
3,3,28,Gemeinschaftspraxis Dr.Klaus Wenzel und Marina...,"{'addressregion': 'Baden-Württemberg', 'addres...",(07931)&nbsp;7761,https://www.11880.com/branchenbuch/bad-mergent...,Unterer Graben 11,Baden-Württemberg,97980,Bad Mergentheim,LocalBusiness_11880.com_September2020,None,info@votteler-wenzel.de,"{'latitude': '49,492303', 'longitude': '9,7704...",http://www.votteler-wenzel.de,//static.11880.com/Portal/images/11880/photo-u...,"[{'opens': ['08:30', '14:30'], 'closes': ['18:...",(07931) 8813,None,None,2020-09-29,DE,NaN,49.492303,9.770486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,079317761
4,4,33,Dipl.-Päd. Nicola Schultheis akademische Sprac...,"{'addressregion': 'Schleswig-Holstein', 'posta...",(0431)&nbsp;2201682,https://www.11880.com/branchenbuch/kiel/111113...,Sophienblatt 88 - 90,Schleswig-Holstein,24114,Kiel,LocalBusiness_11880.com_September2020,None,info@abc-sprachtherapie.de,"{'longitude': '10,126505', 'latitude': '54,310...",https://www.abc-sprachtherapie.de,//static.11880.com/Portal/images/11880/photo-u...,None,None,None,None,2020-09-25,DE,NaN,54.310212,10.126505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04312201682


## Define normalizer for telephone package phonenumbers

In [25]:
def normalizer(entity):
    number = entity["telephone_"]
    address_country = entity["addresscountry"]
    phone_number = phonenumbers.parse(number, address_country)
    return phone_number

## Finally normalizing phone numbers in E.164 format

### Ignore those which can not be identified and replace as ``unknown``

In [26]:
df_clean.reset_index(inplace=True)
phone_objects =[]
#index = []
for row_index in df_clean.index:
    try:
        phone_object =  normalizer(df_clean.iloc[row_index])
        #index.append(row_index)
        phone_objects.append(phone_object)
    except:
        phone_objects.append("unknown")
    
    

In [27]:
len(phone_objects)

86077

In [28]:
df_clean["phone_object"] = pd.Series(phone_objects)

In [29]:
df_clean = df_clean.drop(columns = "index")
df_clean.head()

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object
0,0,7,Fuchsberger Immobilien,"{'addresscountry': 'DE', 'streetaddress': 'Obe...",(06104)&nbsp;43864,https://www.11880.com/branchenbuch/obertshause...,Oberweiser Str. 7,Hessen,63179,Obertshausen,LocalBusiness_11880.com_September2020,None,info@fuchsberger-immobilien.de,"{'latitude': '50,073592', 'longitude': '8,84146'}",http://www.fuchsberger-immobilien.de,[https://cdn.11880.com/fuchsberger-immobilien_...,"[{'closes': '20:00', 'dayofweek': 'Mittwoch', ...",(06104) 499515,None,https://cdn.11880.com/fuchsberger-immobilien_2...,2020-09-18,DE,NaN,50.073592,8.841460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0610443864,Country Code: 49 National Number: 610443864
1,1,19,Ristorante Da Capo,"{'addresslocality': 'Krefeld', 'addresscountry...",(02151)&nbsp;734710,https://www.11880.com/branchenbuch/krefeld/071...,Hülser Markt 10,Nordrhein-Westfalen,47839,Krefeld,LocalBusiness_11880.com_September2020,None,ristorantedacapo@hotmail.de,"{'latitude': '51,372843', 'longitude': '6,5092...",http://dacaporistorante.com,https://cdn.11880.com/ristorante-da-capo_24583...,"[{'opens': ['18:00', '12:00'], 'closes': ['23:...",None,None,https://cdn.11880.com/ristorante-da-capo_24583...,2020-09-27,DE,NaN,51.372843,6.509248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02151734710,Country Code: 49 National Number: 2151734710
2,2,20,Bäckerei Mareis GmbH,"{'addresscountry': 'DE', 'addressregion': 'Bay...",(0871)&nbsp;95389680,https://www.11880.com/branchenbuch/landshut-is...,Ritter-von-Schoch-Str. 20,Bayern,84036,Landshut,LocalBusiness_11880.com_September2020,"{'ratingvalue': '4.00', 'bestrating': '5', 'wo...",info@mareis.com,"{'longitude': '12,179405', 'latitude': '48,546...",https://www.mareis.com,//static.11880.com/Portal/images/11880/photo-u...,"[{'closes': '20:00', 'opens': '06:00', 'dayofw...",None,"{'datemodified': '2020-06-29', 'description': ...",None,2020-09-30,DE,NaN,48.546776,12.179405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,087195389680,Country Code: 49 National Number: 87195389680
3,3,28,Gemeinschaftspraxis Dr.Klaus Wenzel und Marina...,"{'addressregion': 'Baden-Württemberg', 'addres...",(07931)&nbsp;7761,https://www.11880.com/branchenbuch/bad-mergent...,Unterer Graben 11,Baden-Württemberg,97980,Bad Mergentheim,LocalBusiness_11880.com_September2020,None,info@votteler-wenzel.de,"{'latitude': '49,492303', 'longitude': '9,7704...",http://www.votteler-wenzel.de,//static.11880.com/Portal/images/11880/photo-u...,"[{'opens': ['08:30', '14:30'], 'closes': ['18:...",(07931) 8813,None,None,2020-09-29,DE,NaN,49.492303,9.770486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,079317761,Country Code: 49 National Number: 79317761
4,4,33,Dipl.-Päd. Nicola Schultheis akademische Sprac...,"{'addressregion': 'Schleswig-Holstein', 'posta...",(0431)&nbsp;2201682,https://www.11880.com/branchenbuch/kiel/111113...,Sophienblatt 88 - 90,Schleswig-Holstein,24114,Kiel,LocalBusiness_11880.com_September2020,None,info@abc-sprachtherapie.de,"{'longitude': '10,126505', 'latitude': '54,310...",https://www.abc-sprachtherapie.de,//static.11880.com/Portal/images/11880/photo-u...,None,None,None,None,2020-09-25,DE,NaN,54.310212,10.126505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04312201682,Country Code: 49 National Number: 4312201682


In [30]:
unknown_rows = df_clean[df_clean["phone_object"] == "unknown"].index

In [31]:
df_clean = df_clean.drop(unknown_rows)

In [32]:
len(df_clean)

85908

## Check whether phonenumbers are valid

In [33]:
df_valid_numbers = df_clean[df_clean["phone_object"].apply(phonenumbers.is_valid_number)]

In [34]:
len(df_valid_numbers)

78144

## Finally dataset is reduced to roughly 78.k entries
## startet with roughly 500.k entries this reduces size by 80%

## Next step: Format every telephone number into unique E.164 format

In [35]:
#phonenumbers.format_number(df_valid_numbers["phone_object"][0], phonenumbers.PhoneNumberFormat.E164)

In [36]:
df_valid_numbers["E.164 format"] = df_valid_numbers["phone_object"].apply(lambda object: phonenumbers.format_number(object, phonenumbers.PhoneNumberFormat.E164))

<ipython-input-36-8dc4dbf6ff4f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_numbers["E.164 format"] = df_valid_numbers["phone_object"].apply(lambda object: phonenumbers.format_number(object, phonenumbers.PhoneNumberFormat.E164))


In [37]:
len(df_valid_numbers)

78144

In [38]:
df_valid_numbers.head()

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object,E.164 format
0,0,7,Fuchsberger Immobilien,"{'addresscountry': 'DE', 'streetaddress': 'Obe...",(06104)&nbsp;43864,https://www.11880.com/branchenbuch/obertshause...,Oberweiser Str. 7,Hessen,63179,Obertshausen,LocalBusiness_11880.com_September2020,None,info@fuchsberger-immobilien.de,"{'latitude': '50,073592', 'longitude': '8,84146'}",http://www.fuchsberger-immobilien.de,[https://cdn.11880.com/fuchsberger-immobilien_...,"[{'closes': '20:00', 'dayofweek': 'Mittwoch', ...",(06104) 499515,None,https://cdn.11880.com/fuchsberger-immobilien_2...,2020-09-18,DE,NaN,50.073592,8.841460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0610443864,Country Code: 49 National Number: 610443864,+49610443864
1,1,19,Ristorante Da Capo,"{'addresslocality': 'Krefeld', 'addresscountry...",(02151)&nbsp;734710,https://www.11880.com/branchenbuch/krefeld/071...,Hülser Markt 10,Nordrhein-Westfalen,47839,Krefeld,LocalBusiness_11880.com_September2020,None,ristorantedacapo@hotmail.de,"{'latitude': '51,372843', 'longitude': '6,5092...",http://dacaporistorante.com,https://cdn.11880.com/ristorante-da-capo_24583...,"[{'opens': ['18:00', '12:00'], 'closes': ['23:...",None,None,https://cdn.11880.com/ristorante-da-capo_24583...,2020-09-27,DE,NaN,51.372843,6.509248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02151734710,Country Code: 49 National Number: 2151734710,+492151734710
2,2,20,Bäckerei Mareis GmbH,"{'addresscountry': 'DE', 'addressregion': 'Bay...",(0871)&nbsp;95389680,https://www.11880.com/branchenbuch/landshut-is...,Ritter-von-Schoch-Str. 20,Bayern,84036,Landshut,LocalBusiness_11880.com_September2020,"{'ratingvalue': '4.00', 'bestrating': '5', 'wo...",info@mareis.com,"{'longitude': '12,179405', 'latitude': '48,546...",https://www.mareis.com,//static.11880.com/Portal/images/11880/photo-u...,"[{'closes': '20:00', 'opens': '06:00', 'dayofw...",None,"{'datemodified': '2020-06-29', 'description': ...",None,2020-09-30,DE,NaN,48.546776,12.179405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,087195389680,Country Code: 49 National Number: 87195389680,+4987195389680
3,3,28,Gemeinschaftspraxis Dr.Klaus Wenzel und Marina...,"{'addressregion': 'Baden-Württemberg', 'addres...",(07931)&nbsp;7761,https://www.11880.com/branchenbuch/bad-mergent...,Unterer Graben 11,Baden-Württemberg,97980,Bad Mergentheim,LocalBusiness_11880.com_September2020,None,info@votteler-wenzel.de,"{'latitude': '49,492303', 'longitude': '9,7704...",http://www.votteler-wenzel.de,//static.11880.com/Portal/images/11880/photo-u...,"[{'opens': ['08:30', '14:30'], 'closes': ['18:...",(07931) 8813,None,None,2020-09-29,DE,NaN,49.492303,9.770486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,079317761,Country Code: 49 National Number: 79317761,+4979317761
4,4,33,Dipl.-Päd. Nicola Schultheis akademische Sprac...,"{'addressregion': 'Schleswig-Holstein', 'posta...",(0431)&nbsp;2201682,https://www.11880.com/branchenbuch/kiel/111113...,Sophienblatt 88 - 90,Schleswig-Holstein,24114,Kiel,LocalBusiness_11880.com_September2020,None,info@abc-sprachtherapie.de,"{'longitude': '10,126505', 'latitude': '54,310...",https://www.abc-sprachtherapie.de,//static.11880.com/Portal/images/11880/photo-u...,None,None,None,None,2020-09-25,DE,NaN,54.310212,10.126505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04312201682,Country Code:

## After formatting phone numbers into unified format we can group by phone numbers to identify clusters

In [39]:
df_valid_numbers["E.164 format"].value_counts().sort_values().tail(40)

+918043044585     12
+917971377675     12
+918048763569     12
+918047026626     12
+918048883756     13
+918048763701     13
+918048863134     13
+917971390172     13
+918043047438     13
+918043050223     13
+918048400072     13
+918048015696     13
+917971336612     13
+918048970658     13
+918048010210     14
+918046053472     14
+918048083417     14
+5712203000       15
+918042958371     15
+56800510101      15
+918048742935     15
+56800171171      16
+498005424828     16
+917971285353     16
+56800223017      18
+56226787000      19
+918048757741     20
+566006001100     20
+918048728237     21
+918048428092     22
+919898484534     22
+566002007000     23
+918049462300     28
+566007505600     30
+566003632020     32
+918048004430     39
+566006373737     59
+566006201010     63
+566003203000    145
+566007300073    726
Name: E.164 format, dtype: int64

In [40]:
pd.set_option('display.max_columns', 500)
df_valid_numbers[df_valid_numbers["E.164 format"] == "+918047026626"][:5]

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object,E.164 format
65928,67667,127,Nandini Autoparts,{'streetaddress': '317 usha nagar extention na...,+91-8047026626,https://m.indiamart.com/nandiniautoparts/air-c...,317 usha nagar extention narendra tiwari marg ...,Indore,452001,Usha Nagar,LocalBusiness_indiamart.com_September2020,NaN,NaN,"{'latitude': '22.69747', 'longitude': '75.83359'}",None,NaN,NaN,NaN,NaN,NaN,NaN,IN,NaN,22.69747,75.83359,NaN,NaN,Wholesale Trader of Air Cushion Grip Cover off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,918047026626,Country Code: 91 National Number: 8047026626,+918047026626
67264,69123,1727,Nandini Autoparts,{'streetaddress': '317 usha nagar extention na...,+91-8047026626,https://m.indiamart.com/nandiniautoparts/castr...,317 usha nagar extention narendra tiwari marg ...,Indore,452001,Usha Nagar,LocalBusiness_indiamart.com_September2020,NaN,NaN,"{'latitude': '22.69747', 'longitude': '75.83359'}",None,NaN,NaN,NaN,NaN,NaN,NaN,IN,NaN,22.69747,75.83359,NaN,NaN,Wholesale Trader of Castrol Lubricating Oil - ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,918047026626,Country Code: 91 National Number: 8047026626,+918047026626
68323,70279,3000,Nandini Autoparts,{'streetaddress': '317 usha nagar extention na...,+91-8047026626,https://m.indiamart.com/nandiniautoparts/bike-...,317 usha nagar extention narendra tiwari marg ...,Indore,452001,Usha Nagar,LocalBusiness_indiamart.com_September2020,NaN,NaN,"{'latitude': '22.69747', 'longitude': '75.83359'}",None,NaN,NaN,NaN,NaN,NaN,NaN,IN,NaN,22.69747,75.83359,NaN,NaN,Wholesale Trader of Bike Spare Parts - Hero Ho...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,918047026626,Country Code: 91 National Number: 8047026626,+918047026626
69591,71656,4522,Nandini Autoparts,{'streetaddress': '317 usha nagar extention na...,+91-8047026626,https://m.indiamart.com/nandiniautoparts/maint...,317 usha nagar extention narendra tiwari marg ...,Indore,452001,Usha Nagar,LocalBusiness_indiamart.com_September2020,NaN,NaN,"{'latitude': '22.69747', 'longitude': '75.83359'}",None,NaN,NaN,NaN,NaN,NaN,NaN,IN,NaN,22.69747,75.83359,NaN,NaN,Wholesale Trader of Maintenance Spray - Zorrik...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,918047026626,Country Code: 91 National Number: 8047026626,+918047026626
70700,72866,5869,Nandini Autoparts,{'streetaddress': '317 usha nagar extention na...,+91-8047026626,https://m.indiamart.com/nandiniautoparts/,317 usha nagar extention narendra tiwari marg ...,Indore,452001,Usha Nagar,LocalBusiness_indiamart.com_September2020,NaN,NaN,"{'latitude': '22.69747', 'longitude': '75.83359'}",None,NaN,NaN,NaN,NaN,NaN,NaN,IN,NaN,22.69747,75.83359,NaN,NaN,Nandini Autoparts - Wholesale Trader of Engine...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,918047026626,Country Code: 91 National Number: 8047026626,+918047026626


## As one can see from the geo locations this is a *successful* match!

In [41]:
df_valid_numbers[df_valid_numbers["E.164 format"] == "+566006373737"][:5]

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object,E.164 format
35543,35564,214,Cajero automático - Banco de Chile,{'streetaddress': 'Metro Estacion Union Latino...,600 637 3737,https://www.civico.com/lugar/cajero-automatico...,Metro Estacion Union Latinoamericana,Santiago,NaN,SANTIAGO (COMUNA),LocalBusiness_civico.com_September2020,NaN,None,"{'longitude': '-7.0673456E1', 'latitude': '-3....",NaN,NaN,"{'opens': '00:00', 'dayofweek': ['Saturday', '...",NaN,NaN,NaN,NaN,CL,NaN,-33.449423,-70.673456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6006373737,Country Code: 56 National Number: 6006373737,+566006373737
35798,35853,578,Cajero automático - Banco de Chile,"{'streetaddress': 'Metro Estacion Los Heroes',...",600 637 3737,https://www.civico.com/lugar/cajero-automatico...,Metro Estacion Los Heroes,Santiago,NaN,SANTIAGO (COMUNA),LocalBusiness_civico.com_September2020,NaN,None,"{'longitude': '-7.0660228E1', 'latitude': '-3....",NaN,NaN,"{'opens': '00:00', 'dayofweek': ['Saturday', '...",NaN,NaN,NaN,NaN,CL,NaN,-33.446156,-70.660228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6006373737,Country Code: 56 National Number: 6006373737,+566006373737
35821,35881,615,Banco de Chile - San Diego / Arauco,"{'addresslocality': 'SANTIAGO (COMUNA)', 'stre...",6006373737,https://www.civico.com/lugar/banco-de-chile-sa...,San Diego Nº 2029,Santiago,NaN,SANTIAGO (COMUNA),LocalBusiness_civico.com_September2020,NaN,None,"{'latitude': '-3.34721E1', 'longitude': '-7.06...",NaN,NaN,"{'opens': '09:00', 'dayofweek': ['Thursday', '...",NaN,NaN,NaN,NaN,CL,NaN,-33.472100,-70.647800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6006373737,Country Code: 56 National Number: 6006373737,+566006373737
35920,35990,762,Cajero Automático Banco de Chile - Sucursal Ba...,"{'addresslocality': 'SANTIAGO (COMUNA)', 'addr...",6006373737,https://www.civico.com/lugar/cajero-automatico...,Av. Providencia N° 655,Santiago,NaN,SANTIAGO (COMUNA),LocalBusiness_civico.com_September2020,NaN,None,"{'longitude': '-7.06260418E1', 'latitude': '-3...",NaN,NaN,"{'opens': '00:00', 'dayofweek': ['Saturday', '...",NaN,NaN,NaN,NaN,CL,NaN,-33.433624,-70.626042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6006373737,Country Code: 56 National Number: 6006373737,+566006373737
35996,36081,868,Cajero automático - Banco de Chile,"{'addresslocality': 'SAN MIGUEL', 'streetaddre...",600 637 3737,https://www.civico.com/lugar/cajero-automatico...,Metro Estacion Ciudad Del Niño,Santiago,NaN,SAN MIGUEL,LocalBusiness_civico.com_September2020,NaN,None,"{'longitude': '-7.0656653E1', 'latitude': '-3....",NaN,NaN,"{'opens': '00:00', 'dayofweek': ['Saturday', '...",NaN,NaN,NaN,NaN,CL,NaN,-33.509612,-70.656653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6006373737,Country Code: 56 National Number: 6006373737,+566006373737


## Adding the matching telephone numbers in a new column

In [70]:
def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, r = 0):    
    point = [float(i) for i in point]
    idx = tree.query_ball_point(point, r)
    return idx

df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
df_valid_numbers.reset_index(drop=True, inplace=True)
df_valid_numbers['indexValue'] = df_valid_numbers.index
telephoneArray = df_valid_numbers['telephoneNorm'].to_numpy().astype('int64')
fillArray = np.full(len(telephoneArray), 1)
tupleArray = np.array((telephoneArray, fillArray)).T.astype('int64')


# create new column with all matching points
tree = createKDTree(tupleArray)
idx = queryTree(tree, tupleArray[0])

# Search for the closest neighbour in all of the points
df_valid_numbers['MatchingNumbers'] = df_valid_numbers.apply(lambda row: queryTree(tree,[row['telephoneNorm'], 1]), axis=1) 

<ipython-input-70-623d5e3ddf8f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
<ipython-input-70-623d5e3ddf8f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_numbers['indexValue'] = df_valid_numbers.index
<ipython-input-70-623d5e3ddf8f>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [71]:
# filter out the values which only have one value
data = df_valid_numbers[df_valid_numbers['MatchingNumbers'].apply(lambda x: len(x) > 1)]
data

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object,E.164 format,indexValue,MatchingNumbers,telephoneNorm
90,90,222,Physio-Fit Inh. Jenny Wegmann Praxis für Physi...,"{'addresslocality': 'Neunkirchen, Siegerland',...",(02735)&nbsp;6638,https://www.11880.com/branchenbuch/neunkirchen...,Weidenstr. 8,Nordrhein-Westfalen,57290,"Neunkirchen, Siegerland",LocalBusiness_11880.com_September2020,None,praxis@physio-wegmann.de,"{'latitude': '50,783156', 'longitude': '8,0191...",//www.11880.com/branchenbuch/neunkirchen-siege...,//static.11880.com/Portal/images/11880/photo-u...,"[{'opens': '08:00', 'closes': '20:00', 'dayofw...",None,None,None,2020-09-22,DE,NaN,50.783156,8.019114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,027356638,Country Code: 49 National Number: 27356638,+4927356638,90,"[90, 1561]",4927356638
105,105,253,Dr.med. Gordon Splettsen Praktischer Arzt,"{'addresscountry': 'DE', 'addresslocality': 'B...",(06172)&nbsp;22011,https://www.11880.com/branchenbuch/bad-homburg...,Höhestr. 31 - 33,Hessen,61348,Bad Homburg,LocalBusiness_11880.com_September2020,"{'ratingvalue': '5.00', 'bestrating': '5', 'wo...",info@dr-splettsen.eu,"{'latitude': '50,232531', 'longitude': '8,6109...",http://www.dr-splettsen.eu,//static.11880.com/Portal/images/11880/photo-u...,"[{'opens': ['15:00', '09:00'], 'closes': ['17:...",(06172) 683322,"{'datepublished': '2015-02-13', 'datemodified'...",None,2020-09-20,DE,NaN,50.232531,8.610967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0617222011,Country Code: 49 National Number: 617222011,+49617222011,105,"[105, 5714]",49617222011
117,117,276,A + A Elektriker Notdienst,"{'addressregion': 'Nordrhein-Westfalen', 'addr...",(0800)&nbsp;5424828,https://www.11880.com/branchenbuch/kerpen-rhei...,None,Nordrhein-Westfalen,50171,Kerpen,LocalBusiness_11880.com_September2020,None,None,"{'longitude': '6,669989', 'latitude': '50,8630...",//www.11880.com/branchenbuch/kerpen-rheinland/...,//static.11880.com/Portal/images/11880/photo-u...,None,None,None,None,2020-09-25,DE,NaN,50.863091,6.669989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08005424828,Country Code: 49 National Number: 8005424828,+498005424828,117,"[117, 313, 1008, 1064, 2238, 2379, 2457, 2574,...",498005424828
122,122,288,Dr.med.dent. Stefan Reinhardt Zahnarzt,"{'addresscountry': 'DE', 'postalcode': '48143'...",(0251)&nbsp;55155,https://www.11880.com/branchenbuch/muenster-we...,Schorlemerstr. 16,Nordrhein-Westfalen,48143,Münster,LocalBusiness_11880.com_September2020,None,info@implantatzentrum.de,"{'latitude': '51,956132', 'longitude': '7,6297...",https://www.implantatzentrum.de,//static.11880.com/Portal/images/11880/photo-u...,None,(0251) 518945,None,None,2020-09-20,DE,NaN,51.956132,7.629755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,025155155,Country Code: 49 National Number: 25155155,+4925155155,122,"[122, 583, 5300]",4925155155
187,187,404,Dr.med. Ralf Lorenz Facharzt für Frauenheilkun...,"{'addresslocality': 'Braunschweig', 'addressco...",(0531)&nbsp;16024,https://www.11880.com/branchenbuch/braunschwei...,Casparistr. 5 - 6,Niedersachsen,38100,Braunschweig,LocalBusiness_11880.com_September2020,None,None,"{'longitude': '10,524039', 'latitude': '52,266...",//www.11880.com/branchenbuch/braunschweig/1100...,//static.11880.com/Portal/images/11880/photo-u...,"[{'opens': '08:00', 'closes': '16:00', 'dayofw...",(0531) 16025,None,None,2020-09-25,

## Additional Filtering by Geo Location

In [72]:
def calcDifference(pointOne, pointTwo):
    return geopy.distance.great_circle(pointOne, pointTwo).km

def calcDifferenceFromRow (row):
    indexValue = row['indexValue']
    indexPosition = (row[lat], row[lon])
    diffList = []
    for value in row['MatchingGeoPoints']:
        currRow = data.loc[data['indexValue'] == value]
        currIndex = currRow['indexValue'].values[0]
        if currIndex == indexValue:
            diffList.append(-1)
        else:            
            currPosition = (currRow[lat].values[0], currRow[lon].values[0])
            diffList.append(calcDifference(indexPosition, currPosition))
    return diffList


In [73]:
def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, radius = 0.001):    
    point = [float(i) for i in point]
    idx = tree.query_ball_point(point, r=radius)
    return idx
    #idx = tree.query(point, k=neighbours)
    #return idx[1]

# convert to tuples and from string to float
lonArr = data[lon].to_numpy()
latArr = data[lat].to_numpy()
tupleArray = np.array((lonArr, latArr)).T.astype('float32')

data.reset_index(drop=True, inplace=True)
data['indexValue'] = data.index

# create new column with all matching points
tree = createKDTree(tupleArray)
idx = queryTree(tree, tupleArray[0])

# Search for the closest neighbour in all of the points
data['MatchingGeoPoints'] = data.apply(lambda row: queryTree(tree,[row[lon], row[lat]]), axis=1) 

# Keep those that have one or more matches withing the radius
data = data[data['MatchingGeoPoints'].apply(lambda x: len(x) > 1)]

# Calculate the difference in km between those
data['Difference'] = data.apply(lambda row: calcDifferenceFromRow(row), axis=1) 

<ipython-input-73-4b38e49197bf>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['indexValue'] = data.index
<ipython-input-73-4b38e49197bf>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MatchingGeoPoints'] = data.apply(lambda row: queryTree(tree,[row[lon], row[lat]]), axis=1)
<ipython-input-73-4b38e49197bf>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [74]:
data.head(5)

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object,E.164 format,indexValue,MatchingNumbers,telephoneNorm,MatchingGeoPoints,Difference
1,105,253,Dr.med. Gordon Splettsen Praktischer Arzt,"{'addresscountry': 'DE', 'addresslocality': 'B...",(06172)&nbsp;22011,https://www.11880.com/branchenbuch/bad-homburg...,Höhestr. 31 - 33,Hessen,61348,Bad Homburg,LocalBusiness_11880.com_September2020,"{'ratingvalue': '5.00', 'bestrating': '5', 'wo...",info@dr-splettsen.eu,"{'latitude': '50,232531', 'longitude': '8,6109...",http://www.dr-splettsen.eu,//static.11880.com/Portal/images/11880/photo-u...,"[{'opens': ['15:00', '09:00'], 'closes': ['17:...",(06172) 683322,"{'datepublished': '2015-02-13', 'datemodified'...",None,2020-09-20,DE,NaN,50.232531,8.610967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0617222011,Country Code: 49 National Number: 617222011,+49617222011,1,"[105, 5714]",49617222011,"[1, 105]","[-1, 0.0]"
3,122,288,Dr.med.dent. Stefan Reinhardt Zahnarzt,"{'addresscountry': 'DE', 'postalcode': '48143'...",(0251)&nbsp;55155,https://www.11880.com/branchenbuch/muenster-we...,Schorlemerstr. 16,Nordrhein-Westfalen,48143,Münster,LocalBusiness_11880.com_September2020,None,info@implantatzentrum.de,"{'latitude': '51,956132', 'longitude': '7,6297...",https://www.implantatzentrum.de,//static.11880.com/Portal/images/11880/photo-u...,None,(0251) 518945,None,None,2020-09-20,DE,NaN,51.956132,7.629755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,025155155,Country Code: 49 National Number: 25155155,+4925155155,3,"[122, 583, 5300]",4925155155,"[3, 8, 95]","[-1, 0.0, 0.0]"
4,187,404,Dr.med. Ralf Lorenz Facharzt für Frauenheilkun...,"{'addresslocality': 'Braunschweig', 'addressco...",(0531)&nbsp;16024,https://www.11880.com/branchenbuch/braunschwei...,Casparistr. 5 - 6,Niedersachsen,38100,Braunschweig,LocalBusiness_11880.com_September2020,None,None,"{'longitude': '10,524039', 'latitude': '52,266...",//www.11880.com/branchenbuch/braunschweig/1100...,//static.11880.com/Portal/images/11880/photo-u...,"[{'opens': '08:00', 'closes': '16:00', 'dayofw...",(0531) 16025,None,None,2020-09-25,DE,NaN,52.266339,10.524039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,053116024,Country Code: 49 National Number: 53116024,+4953116024,4,"[187, 4122]",4953116024,"[4, 71]","[-1, 0.0]"
6,355,747,WBSeminare & NLP,"{'addresslocality': 'Aachen', 'addresscountry'...",(0241)&nbsp;95497690,https://www.11880.com/branchenbuch/aachen/0619...,Kreuzerdriesch 36,Nordrhein-Westfalen,52076,Aachen,LocalBusiness_11880.com_September2020,None,w.brylla@wbseminare-nlp.de,"{'latitude': '50,740687', 'longitude': '6,1294...",https://www.wbseminare-nlp.de,//static.11880.com/Portal/images/11880/photo-u...,None,None,None,None,2020-09-28,DE,NaN,50.740687,6.129467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024195497690,Country Code: 49 National Number: 24195497690,+4924195497690,6,"[355, 76239]",4924195497690,"[6, 16709]","[-1, 0.004592890569090546]"
7,539,1085,Maren Adam Kieferorthopädin,"{'addressregion': 'Schleswig-Holstein', 'addre...",(0431)&nbsp;380049-0,https://www.11880.com/branchenbuch/kiel/111912...,Friedrichsorter Str. 10,Schleswig-Holstein,24159,Kiel,LocalBusiness_11880.com_September2020,None,info@cmd-ambulanz.de,"{'latitude': '54,396663', 'longitude': '10,172...",https://kieferorthopaede-kiel.de,//static.11880.com/Portal/images/11880/photo-u...,"[{'closes': ['17:30',

In [75]:
len(data)

15355

In [76]:
data.loc[data['indexValue'] == 101]

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object,E.164 format,indexValue,MatchingNumbers,telephoneNorm,MatchingGeoPoints,Difference
101,5539,11647,Dipl.-Psychologin Heike Bollig-Großerüschkamp ...,"{'addresscountry': 'DE', 'streetaddress': 'Ehr...",(02234)&nbsp;801091,https://www.11880.com/branchenbuch/pulheim/111...,Ehrenfriedstr. 38,Nordrhein-Westfalen,50259,Pulheim,LocalBusiness_11880.com_September2020,None,None,"{'longitude': '6,782011', 'latitude': '50,9606...",//www.11880.com/branchenbuch/pulheim/111114204...,//static.11880.com/Portal/images/11880/photo-u...,None,None,None,None,2020-09-29,DE,NaN,50.960615,6.782011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02234801091,Country Code: 49 National Number: 2234801091,+492234801091,101,"[1393, 5529]",492234801091,"[21, 101]","[0.0, -1]"


In [77]:
data.loc[data['indexValue'] == 3]

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object,E.164 format,indexValue,MatchingNumbers,telephoneNorm,MatchingGeoPoints,Difference
3,122,288,Dr.med.dent. Stefan Reinhardt Zahnarzt,"{'addresscountry': 'DE', 'postalcode': '48143'...",(0251)&nbsp;55155,https://www.11880.com/branchenbuch/muenster-we...,Schorlemerstr. 16,Nordrhein-Westfalen,48143,Münster,LocalBusiness_11880.com_September2020,None,info@implantatzentrum.de,"{'latitude': '51,956132', 'longitude': '7,6297...",https://www.implantatzentrum.de,//static.11880.com/Portal/images/11880/photo-u...,None,(0251) 518945,None,None,2020-09-20,DE,NaN,51.956132,7.629755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,025155155,Country Code: 49 National Number: 25155155,+4925155155,3,"[122, 583, 5300]",4925155155,"[3, 8, 95]","[-1, 0.0, 0.0]"


In [78]:
data.loc[data['indexValue'] == 8]

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object,E.164 format,indexValue,MatchingNumbers,telephoneNorm,MatchingGeoPoints,Difference
8,586,1187,Dr.med.dent. Bernhard Drüke Zahnarzt,"{'addresscountry': 'DE', 'postalcode': '48143'...",(0251)&nbsp;55155,https://www.11880.com/branchenbuch/muenster-we...,Schorlemerstr. 16,Nordrhein-Westfalen,48143,Münster,LocalBusiness_11880.com_September2020,None,info@implantatzentrum.de,"{'latitude': '51,956132', 'longitude': '7,6297...",http://drdrüke.de,//static.11880.com/Portal/images/11880/photo-u...,None,(0251) 518945,None,None,2020-09-28,DE,NaN,51.956132,7.629755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,025155155,Country Code: 49 National Number: 25155155,+4925155155,8,"[122, 583, 5300]",4925155155,"[3, 8, 95]","[0.0, -1, 0.0]"


In [81]:
data.loc[data['indexValue'] == 95]

,level_0,row_id,name,address,telephone,page_url,streetaddress,addressregion,postalcode,addresslocality,table id,aggregaterating,email,geo,url,image,openinghoursspecification,faxnumber,review,logo,datemodified,addresscountry,postofficeboxnumber,latitude,longitude,pricerange,hasmap,description,sameas,citystatezip,telephone_address,price,serialnumber,openinghours,areaserved,paymentaccepted,brand,publicaccess,fax,interactioncount,hasoffercatalog,currenciesaccepted,location,employee,knowsabout,legalname,telephone_,phone_object,E.164 format,indexValue,MatchingNumbers,telephoneNorm,MatchingGeoPoints,Difference
95,5309,11157,Dr.med.dent. Josef Janzen Zahnarzt,"{'addresscountry': 'DE', 'postalcode': '48143'...",(0251)&nbsp;55155,https://www.11880.com/branchenbuch/muenster-we...,Schorlemerstr. 16,Nordrhein-Westfalen,48143,Münster,LocalBusiness_11880.com_September2020,None,info@implantatzentrum.de,"{'latitude': '51,956132', 'longitude': '7,6297...",https://www.implantatzentrum.de,https://cdn.11880.com/dr-med-dent-josef-janzen...,None,(0251) 518945,None,https://cdn.11880.com/dr-med-dent-josef-janzen...,2020-09-25,DE,NaN,51.956132,7.629755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,025155155,Country Code: 49 National Number: 25155155,+4925155155,95,"[122, 583, 5300]",4925155155,"[3, 8, 95]","[0.0, 0.0, -1]"
